In [ ]:
### Final version in the VSCODE

In [1]:
import pandas as pd
import plotly.graph_objects as go
import os

In [2]:
##########################################
# 1.Combine the Colman's Assembly and Viridian Assembly
##########################################

# After /nfs/research/goldman/zihao/code/EBI_INTER/Project_2_vircolComp/2.mapleComp_P3/Pg_1_dataPrepration.py
Folder_col = "/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/Folder_mapleOutput/COL_output_modified.txt"
Folder_vir = "/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/Folder_mapleOutput/VIR_output_modified.txt"

df_col = pd.read_csv(Folder_col, sep='\t')
df_vir = pd.read_csv(Folder_vir, sep='\t')

df_vir['VIR'] = 1
df_col['COL'] = 1
merged_df = pd.merge(df_vir, df_col, on=['ID','Position'], how='outer')
df_filled = merged_df.fillna(0)

df_filled['COL'] = pd.to_numeric(df_filled['COL'], errors='coerce').astype(int)
df_filled['VIR'] = pd.to_numeric(df_filled['VIR'], errors='coerce').astype(int)
df_filled

,ID,Position,VIR,COL
0,ERR4590003,3078,1,1
1,ERR4590003,18425,1,1
2,ERR4589145,5941,1,1
3,ERR4589114,21715,1,1
4,ERR4589835,18436,1,0
...,...,...,...,...
51419,ERR4423816,23895,0,1
51420,ERR4423892,28275,0,1
51421,ERR4425059,27008,0,1
51422,SRR21662653,8984,0,1


In [3]:
Folder_path = "/nfs/research/goldman/zihao/Datas/p2_comp_viridian/3_combination/ERR7269308.txt"
df_comb = pd.read_csv(Folder_path, sep='\t')[['position', 'nucleotide_martin', 'nucleotide_origin', 
                                           'label_mar', 'label_ori']]
df_comb

,position,nucleotide_martin,nucleotide_origin,label_mar,label_ori
0,1,-,n,1,1
1,2,-,n,1,1
2,3,-,n,1,1
3,4,-,n,1,1
4,5,-,n,1,1
...,...,...,...,...,...
29898,29899,-,n,1,1
29899,29900,-,n,1,1
29900,29901,-,n,1,1
29901,29902,-,n,1,1


In [ ]:
import pandas as pd
import os

def combine_assemblies(folder_col, folder_vir, folder_data, output_dir, not_found_ids_file):

    # 1.Combine the Colman's Assembly and Viridian Assembly
    df_col = pd.read_csv(folder_col, sep='\t')
    df_vir = pd.read_csv(folder_vir, sep='\t')

    df_vir['VIR'] = 1
    df_col['COL'] = 1
    merged_df = pd.merge(df_vir, df_col, on=['ID','Position'], how='outer')
    df_filled = merged_df.fillna(0)

    df_filled['COL'] = pd.to_numeric(df_filled['COL'], errors='coerce').astype(int)
    df_filled['VIR'] = pd.to_numeric(df_filled['VIR'], errors='coerce').astype(int)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Store not found IDs
    not_found_ids = []

    for index, row in df_filled.iterrows():
        file_path = f'{folder_data}/{row["ID"]}.txt'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, sep='\t')[['position', 'nucleotide_martin', 'nucleotide_origin', 'label_mar', 'label_ori']]
            if row['Position'] in df['position'].values:
                df.loc[df['position'] == row['Position'], 'VIR'] = row['VIR']
                df.loc[df['position'] == row['Position'], 'COL'] = row['COL']
                df.to_csv(os.path.join(output_dir, f'{row["ID"]}.txt'), index=False, sep='\t')
            else:
                not_found_ids.append(row["ID"])
        else:
            not_found_ids.append(row["ID"])

    # Check if parent folder of not_found_ids_file exists and create if it does not
    if not os.path.exists(os.path.dirname(not_found_ids_file)):
        os.makedirs(os.path.dirname(not_found_ids_file))

    # Save not found IDs to a file
    with open(not_found_ids_file, 'w') as f:
        for item in not_found_ids:
            f.write("%s\n" % item)

    # Print the location of not_found_ids_file
    print(f"The not found IDs file has been saved to: {not_found_ids_file}")

    # Print the number of not found IDs
    print(f"The number of not found IDs: {len(not_found_ids)}")

    # Iterate over all files in the folder_data directory
    for filename in os.listdir(folder_data):
        file_id = filename.split('.')[0]  # Get the ID from the filename
        if file_id not in df_filled['ID'].values:  # If ID is not in df_filled
            df = pd.read_csv(f'{folder_data}/{filename}', sep='\t')
            df['COL'] = np.nan  # Add 'COL' column with NaN
            df['VIR'] = np.nan  # Add 'VIR' column with NaN
            df.to_csv(os.path.join(output_dir, filename), index=False, sep='\t')  # Save to output_dir

folder_col = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/Folder_mapleOutput/COL_output_modified.txt'
folder_vir = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/Folder_mapleOutput/VIR_output_modified.txt'
folder_data = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P2/Folder_2_combination/folderData_combination'
output_dir = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/folderData_addError'
not_found_ids_file = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/notFound_idsError.txt'

combine_assemblies(folder_col, folder_vir, folder_data, output_dir, not_found_ids_file)

In [32]:
df = pd.read_csv('/nfs/research/goldman/zihao/TEST/folderData_addError/SRR22291379.txt',sep='\t')
df[~pd.isna(df['COL'])]

,position,nucleotide_martin,nucleotide_origin,label_mar,label_ori,VIR,COL
20220,20221,a,a,0,0,1.0,1.0


In [36]:
import pandas as pd
import os

directory = '/nfs/research/goldman/zihao/TEST/folderData_addError'
found = False
count = 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, sep='\t')
        df_filtered = df[~pd.isna(df['COL'])]
        
        if not df_filtered.empty:
            count +=1
            # print(f"Found suitable data in file: {filename}")
            found = True
            # break
print(count)
if not found:
    print("No suitable data found in any file.")


FileNotFoundError: [Errno 2] No such file or directory: '/nfs/research/goldman/zihao/TEST/folderData_addError/SRR22037852.txt'

In [27]:
import os

directory = '/nfs/research/goldman/zihao/TEST/folderData_addError'

# 获取所有文件名
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# 打印文件个数
print(f"文件夹中有 {len(files)} 个文件。")


文件夹中有 75 个文件。


In [ ]:
import pandas as pd
import os
import argparse
import numpy as np

def combine_assemblies(folder_col, folder_vir, folder_data, output_dir, not_found_ids_file, chunk_num):
    
    # 1.Combine the Colman's Assembly and Viridian Assembly
    df_col = pd.read_csv(folder_col, sep='\t')
    df_vir = pd.read_csv(folder_vir, sep='\t')

    df_vir['VIR'] = 1
    df_col['COL'] = 1
    merged_df = pd.merge(df_vir, df_col, on=['ID','Position'], how='outer')
    df_filled = merged_df.fillna(0)

    df_filled['COL'] = pd.to_numeric(df_filled['COL'], errors='coerce').astype(int)
    df_filled['VIR'] = pd.to_numeric(df_filled['VIR'], errors='coerce').astype(int)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # ======================= Split the df_filled into 10 chunks =======================
    # Split the df_filled into 10 chunks
    chunks = np.array_split(df_filled, 1000)

    # Only process the chunk specified by chunk_num
    chunk = chunks[chunk_num]
    # ==================================================
    
    not_found_ids = []

    for index, row in chunk.iterrows():
        file_path = f'{folder_data}/{row["ID"]}.txt'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, sep='\t')[['position', 'nucleotide_martin', 'nucleotide_origin', 'label_mar', 'label_ori']]
            if row['Position'] in df['position'].values:
                df.loc[df['position'] == row['Position'], 'VIR'] = row['VIR']
                df.loc[df['position'] == row['Position'], 'COL'] = row['COL']
                df.to_csv(os.path.join(output_dir, f'{row["ID"]}.txt'), index=False, sep='\t')
            else:
                not_found_ids.append(row["ID"])
        else:
            not_found_ids.append(row["ID"])

    # Check if parent folder of not_found_ids_file exists and create if it does not
    if not os.path.exists(os.path.dirname(not_found_ids_file)):
        os.makedirs(os.path.dirname(not_found_ids_file))

    # Save not found IDs to a file
    with open(not_found_ids_file, 'w') as f:
        for item in not_found_ids:
            f.write("%s\n" % item)

    # Print the location of not_found_ids_file
    print(f"The not found IDs file has been saved to: {not_found_ids_file}")

    # Print the number of not found IDs
    print(f"The number of not found IDs: {len(not_found_ids)}")

    # Iterate over all files in the folder_data directory
    for filename in os.listdir(folder_data):
        file_id = filename.split('.')[0]  # Get the ID from the filename
        if file_id not in df_filled['ID'].values:  # If ID is not in df_filled
            df = pd.read_csv(f'{folder_data}/{filename}', sep='\t')
            df['COL'] = np.nan  # Add 'COL' column with NaN
            df['VIR'] = np.nan  # Add 'VIR' column with NaN
            df.to_csv(os.path.join(output_dir, filename), index=False, sep='\t')  # Save to output_dir
            
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Process a chunk of the dataframe.')
    parser.add_argument('chunk_num', type=int, help='Chunk number (0-9) to process')
    args = parser.parse_args()

    folder_col = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/Folder_mapleOutput/COL_output_modified.txt'
    folder_vir = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/Folder_mapleOutput/VIR_output_modified.txt'
    
    # temp
    folder_data = '/nfs/research/goldman/zihao/TEST/folderData_combTEST'

    output_dir = '/nfs/research/goldman/zihao/TEST/folderData_addError'
    not_found_ids_file = '/nfs/research/goldman/zihao/TEST/notFound_idsError.txt'
    #output_dir = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/folderData_addError'
    #not_found_ids_file = '/nfs/research/goldman/zihao/Datas/p2_compViridian_P3/notFound_idsError.txt'
    # temp
    combine_assemblies(folder_col, folder_vir, folder_data, output_dir, not_found_ids_file, args.chunk_num)